<a href="https://colab.research.google.com/github/m-triassi/ai-projects-472/blob/main/Project%203/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Dependencies
from gensim.downloader import load
from gensim.models import word2vec
from gensim.similarities import SoftCosineSimilarity
import pandas as pd
import csv
import random # XD RaWR L0L im so R4nDOM

In [ ]:
# Grab dataset csv
!wget https://raw.githubusercontent.com/m-triassi/ai-projects-472/main/Project%203/synonyms.csv?token=ACIA3M7EM7SLE5PLCHH3CRTBWZZ2S -O synonyms.csv

--2021-12-07 04:11:27--  https://raw.githubusercontent.com/m-triassi/ai-projects-472/main/Project%203/synonyms.csv?token=ACIA3M7EM7SLE5PLCHH3CRTBWZZ2S
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4415 (4.3K) [text/plain]
Saving to: ‘synonyms.csv’

synonyms.csv        100%[===================>]   4.31K  --.-KB/s    in 0s      

2021-12-07 04:11:27 (47.5 MB/s) - ‘synonyms.csv’ saved [4415/4415]



In [ ]:
headers = ["question","answer","A","B","C","D"]
synonyms = pd.read_csv("/content/synonyms.csv", header=None, names=headers)[1:]

In [ ]:
# Define similarity run method
# accepts model to be used
def similarity_all(model: word2vec) -> list:
  details = []
  ## iterates over synonyms
  for s in synonyms.iterrows():
    ## Catch if the word is not present, and perfrom a guess.
    guesses = {}
    label = ""
    guess = ""
    if not s[1]["question"] in model.vocab:
      guess = s[1][random.choice(["A", "B", "C", "D"])]
      label = "guess"
  
    for q in ["A", "B", "C", "D"]:
      try:
        guesses[q] = (model.similarity(s[1]["question"], s[1][q]))
      except KeyError:
        continue

    if guesses:
      # sort the guesses by their value
      max_key = max(guesses, key=guesses.get)
      guess = s[1][max_key]
    elif (label is not "guess"):
      # none of the guess-words were in the vocab, just guess something..
      guess = model.most_similar(positive=s[1]["question"], topn=1)[0]
      label = "guess"
    
    if label is not "guess" and guess == s[1]["answer"]:
      label = "correct"
    elif label is not "guess":
      label = "wrong"

    # build the details array
    details.append([s[1]["question"], s[1]["answer"], guess, label])
  
  return details


In [ ]:
# Build analysis for a particular run function
## could just read the details csv and generate a new line in analysis.csv
def analyze_details(model: word2vec, details: pd.DataFrame, model_name):
  analysis = []
  correct_count = 0
  guess_count = 0
  for d in details.iterrows():
    correct_count += d[1]['label'] == "correct"
    guess_count += d[1]['label'] == "guess"

  V = 80 - guess_count
  analysis_header = ['model_name', 'vocab_size', 'C', 'V', 'accuracy']
  analysis_data = [model_name, len(model.vocab), correct_count, V, correct_count/V]
  pd.DataFrame([analysis_data], columns=analysis_header).to_csv('analysis.csv', mode="a", header=False)

In [ ]:
def generate_reports(model: word2vec, label):
  details = similarity_all(model)
  detail_headers = ["question-word", "answer-word", "guess-word", "label"]
  detail_data = pd.DataFrame(details, columns=detail_headers)

  detail_data.to_csv(f"{label}-details.csv")
  analyze_details(model, detail_data, label)

## Task 1
Load the Word2Vec google news pretrained model, evaluate performance using gold standard test.

In [ ]:
# Load google-news-300 pre-trained model
model = load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Test Word2Vec model, save details, generate analysis
generate_reports(model, "word2vec-google-news-300")

## Task 2
Repeat analysis for 4 more models. 2 with same embedding but different corpus, and 2 with differnt embedding but same corpus.

Leverage functions written for task 1.

Models: 
- glove-twitter-50 vs glove-wiki-gigaword-50
- glove-wiki-gigaword-100 vs glove-wiki-gigaword-300 

In [ ]:
model = load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [ ]:
# Test Word2Vec model, save details, generate analysis
generate_reports(model, "glove-twitter-50")

In [ ]:
model = load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
# Test Word2Vec model, save details, generate analysis
generate_reports(model, "glove-wiki-gigaword-50")

In [ ]:
model = load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
# Test Word2Vec model, save details, generate analysis
generate_reports(model, "glove-wiki-gigaword-300")

In [ ]:
model = load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# Test Word2Vec model, save details, generate analysis
generate_reports(model, "glove-wiki-gigaword-100")